In [7]:
import polars as pl 
import pandas as pd 
import os
from ELO_Engine import Elo_Engine

In [20]:
import polars as pl
import os

def limpiar_csv(año1, año2):
    route = f"Ligas_Resultados_CSV/laliga_{año1}_{año2}.csv"

    if not os.path.exists(route):
        print(f"No encontrado: {route}")
        return

    df = pl.read_csv(route)
    df = df.drop(["Fecha", "Hora", "Estadio", "Resultado"])
    df = df.rename({"Asistencia": "Resultado"})

    Jornada = df["Jornada"].to_list()
    Resultado = df["Resultado"].to_list()

    nueva_jornada = []
    nuevo_resultado = []
    nuevo_ganador = []
    ultimo_year = []
    diff_goles = []

    # Limpiar columna Jornada
    for i in Jornada:
        nueva_colJor = i.split(" ")[1].strip()
        nueva_jornada.append(nueva_colJor)

    df = df.with_columns(pl.Series("Jornada", nueva_jornada))

    # Limpiar columna Resultado
    for i in Resultado:
        nueva_colRes = i.split('(')[0].strip()
        nuevo_resultado.append(nueva_colRes)
    df = df.with_columns(pl.Series("Resultado", nuevo_resultado))

    # Calcular ganador y diferencia de goles
    for idx, i in enumerate(Resultado):
        score_str = i.split('(')[0].strip()
        local_score = int(score_str.split(':')[0].strip())
        visitante_score = int(score_str.split(':')[1].strip())
        if local_score > visitante_score:
            ganador = df["Local"][idx]
        elif local_score < visitante_score:
            ganador = df["Visitante"][idx]
        else:
            ganador = "Empate"
        nuevo_ganador.append(ganador)
        diff_gol = abs(local_score - visitante_score)
        diff_goles.append(diff_gol)

    df = df.with_columns(pl.Series("Differencia Gol", diff_goles))
    df = df.with_columns(pl.Series("Ganador", nuevo_ganador))

    # Extraemos el año
    for i in Jornada:
        if '(' in i and ')' in i:
            year_part = i.split('(')[1].split(')')[0].strip()
            ultimo_year.append(year_part)
        else:
            ultimo_year.append("")
    df = df.with_columns(pl.Series("Año", ultimo_year))

    # Convertimos la columna Jornada a int
    df = df.with_columns(
        pl.col("Jornada").cast(pl.Int64)
    )

    salida = f"CLEAN_Ligas_Resultados_CSV/laliga_{año1}_{año2}_clean.csv"
    df.write_csv(salida)
    print(f"Guardado: {salida}")


# 
for año1 in range(1995, 1996):
    año2 = año1 + 1
    limpiar_csv(año1, año2)


Guardado: CLEAN_Ligas_Resultados_CSV/laliga_1995_1996_clean.csv


In [19]:
routex = f"CLEAN_Ligas_Resultados_CSV/laliga_2024_2025_clean.csv"
df = pl.read_csv(routex)
df

Jornada,Local,Visitante,Resultado,Differencia Gol,Ganador,Año
i64,str,str,str,i64,str,str
1,"""Athletic""","""Getafe""","""1:1""",0,"""Empate""","""2024/2025"""
1,"""Betis""","""Girona""","""1:1""",0,"""Empate""","""2024/2025"""
1,"""Celta""","""Alavés""","""2:1""",1,"""Celta""","""2024/2025"""
1,"""Las Palmas""","""Sevilla""","""2:2""",0,"""Empate""","""2024/2025"""
1,"""Osasuna""","""Leganés""","""1:1""",0,"""Empate""","""2024/2025"""
…,…,…,…,…,…,…
38,"""Getafe""","""Celta""","""1:2""",1,"""Celta""","""2024/2025"""
38,"""Rayo""","""Mallorca""","""0:0""",0,"""Empate""","""2024/2025"""
38,"""Girona""","""Atlético""","""0:4""",4,"""Atlético""","""2024/2025"""


In [ ]:
# Lee el archivo elo.csv y añade los equipos que no están en elo.csv
elo_path = f"ELO/ELO_{año1}_{año2}.csv"

if not os.path.exists(elo_path):
    # Crea el DataFrame vacío con las columnas necesarias
    elo_df = pd.DataFrame(columns=["Equipo", "elo"])
    elo_df.to_csv(elo_path, index=False)
else:
    elo_df = pd.read_csv(elo_path)

unique = df["Local"].unique().to_list()
equipos_elo = set(elo_df["Equipo"])


nuevos_equipos = []

for equipo in unique:
    if equipo not in equipos_elo:
        nuevos_equipos.append({"Equipo": equipo, "elo": 1000})

if nuevos_equipos:
    nuevos_df = pd.DataFrame(nuevos_equipos)
    elo_df = pd.concat([elo_df, nuevos_df], ignore_index=True)
